### Task 5: Strategy Backtesting

Simulate the performance of the optimized portfolio against a simple benchmark (60% SPY / 40% BND) over the last year of data.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.data_manager import DataManager
from src.task5_backtest import BacktestOrchestrator

plt.style.use('seaborn-v0_8')
%matplotlib inline


### 1) Data and Backtest Window

Use last year as backtest window (Aug 1, 2024 — Jul 31, 2025).


In [ ]:
START_DATE = '2015-07-01'
END_DATE = '2025-07-31'
BACKTEST_START = '2024-08-01'
BACKTEST_END = '2025-07-31'

TICKERS = ['TSLA', 'BND', 'SPY']

dm = DataManager(data_source='yfinance')
raw = dm.fetch_data(TICKERS, start_date=START_DATE, end_date=END_DATE, frequency='1d')
prices = pd.DataFrame({
    'TSLA': raw['TSLA']['Adj Close'],
    'BND': raw['BND']['Adj Close'],
    'SPY': raw['SPY']['Adj Close'],
}).dropna()

orch = BacktestOrchestrator(initial_capital=100000, transaction_costs=0.001)
windowed_returns = orch.prepare_returns(prices, BACKTEST_START, BACKTEST_END)
prices.tail(), windowed_returns.head()


### 2) Strategy and Benchmark Weights

- Strategy: Use initial optimal weights from Task 4 (enter them or load from saved output). For this template, set a reasonable example.
- Benchmark: 60% SPY / 40% BND (TSLA weight = 0).


In [ ]:
# Example strategy weights (replace with Task 4 optimal weights)
strategy_weights_map = {'TSLA': 0.3, 'BND': 0.3, 'SPY': 0.4}

benchmark_weights_map = {'TSLA': 0.0, 'BND': 0.4, 'SPY': 0.6}

strategy_weights_df = orch.make_constant_weights(windowed_returns.index, strategy_weights_map)
benchmark_weights_df = orch.make_constant_weights(windowed_returns.index, benchmark_weights_map)

strategy_weights_df.head(), benchmark_weights_df.head()


### 3) Run Backtests (Monthly Rebalance)


In [ ]:
res_strategy = orch.run(windowed_returns, strategy_weights_df, rebalance_frequency='monthly')
res_benchmark = orch.run(windowed_returns, benchmark_weights_df, rebalance_frequency='monthly')

res_strategy.performance_metrics, res_benchmark.performance_metrics


### 4) Plot Cumulative Returns vs Benchmark


In [ ]:
fig, ax = plt.subplots(figsize=(12,5))

# Compute cumulative curves
def cumcurve(portfolio_values: pd.DataFrame) -> pd.Series:
    return portfolio_values['portfolio_value'] / portfolio_values['portfolio_value'].iloc[0]

cum_strategy = cumcurve(res_strategy.portfolio_values)
cum_benchmark = cumcurve(res_benchmark.portfolio_values)

cum_strategy.plot(ax=ax, label='Strategy')
cum_benchmark.plot(ax=ax, label='Benchmark (60/40)')

ax.set_title('Cumulative Returns: Strategy vs Benchmark')
ax.legend();


### 5) Summary and Conclusion

- Show total return and Sharpe for both, and provide a brief interpretation.


In [ ]:
summary = pd.DataFrame([
    {
        'Name': 'Strategy',
        'TotalReturn': res_strategy.performance_metrics['total_return'],
        'Sharpe': res_strategy.performance_metrics['sharpe_ratio'],
        'AnnReturn': res_strategy.performance_metrics['annualized_return'],
        'AnnVol': res_strategy.performance_metrics['annualized_volatility'],
    },
    {
        'Name': 'Benchmark',
        'TotalReturn': res_benchmark.performance_metrics['total_return'],
        'Sharpe': res_benchmark.performance_metrics['sharpe_ratio'],
        'AnnReturn': res_benchmark.performance_metrics['annualized_return'],
        'AnnVol': res_benchmark.performance_metrics['annualized_volatility'],
    },
])
summary.set_index('Name')
